In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install catboost
!pip install scikit-optimize

In [ ]:
import pandas as pd
from time import time
import pprint
import joblib
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier , AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
from catboost import CatBoostClassifier
from google.colab import files
# Model selection
from sklearn.model_selection import StratifiedKFold
import seaborn as sn
from sklearn.impute import KNNImputer
import numpy as np
# Metrics
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer

from lightgbm import LGBMClassifier

## PET CATEGORY

In [ ]:
train = pd.read_csv(r"/content/gdrive/My Drive/Pet adoption/train.csv")
X_test = pd.read_csv(r"/content/gdrive/My Drive/Pet adoption/test.csv")

In [ ]:
train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [ ]:
X_train = train.iloc[:,:9]
y_train1 = train.iloc[:,9:10]
y_train2 = train.iloc[:,10:]

In [ ]:
X_train['condition'] = X_train['condition'].fillna(-1)
X_test['condition'] = X_test['condition'].fillna(-1)

In [ ]:
X_train['label'] = 'train'
X_test['label'] = 'test'
df_ = pd.concat([X_train,X_test])
dummies = pd.get_dummies(df_['color_type'])
dummies1 = pd.get_dummies(df_['condition'])
NEW = pd.concat([df_,dummies],axis='columns')
NEW = pd.concat([NEW,dummies1],axis='columns')

In [ ]:
NEW['length(cm)'] = NEW['length(m)']*100
NEW.drop('length(m)',axis = 1,inplace = True)
NEW['lh'] = (NEW['length(cm)']*NEW['height(cm)']) / 100
NEW['X1_X2'] = NEW['X1']*NEW['X2']
NEW['l_zero'] = np.where(NEW['length(cm)'] == 0,1,0)

In [ ]:
NEW['1stnum'] = NEW['pet_id'].str[5:6].astype(str).astype(int)
NEW['1st2num'] = NEW['pet_id'].str[5:7].astype(str).astype(int)

In [ ]:
NEW['issue_date'] = pd.to_datetime(NEW['issue_date'])
NEW['listing_date'] =pd.to_datetime(NEW['listing_date'])

NEW['issue_date_year'] = NEW['issue_date'].dt.year
NEW['issue_date_month'] = NEW['issue_date'].dt.month
NEW['issue_date_week'] = NEW['issue_date'].dt.week
NEW['issue_date_day'] = NEW['issue_date'].dt.day

NEW['listing_date_year'] = NEW['listing_date'].dt.year
NEW['listing_date_month'] = NEW['listing_date'].dt.month
NEW['listing_date_week'] = NEW['listing_date'].dt.week
NEW['listing_date_day'] = NEW['listing_date'].dt.day
NEW['listing_date_hour'] = NEW['listing_date'].dt.hour
NEW['difference_years'] = NEW['listing_date_year'] - NEW['issue_date_year']

In [ ]:
x=[]
for d in NEW['listing_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)
NEW['modified_listing_date']=x

x=[]
for d in NEW['issue_date']:
    y=d.year+(d.month/12.0)+(d.day/365.0)
    x.append(y)

NEW['modified_issue_date']=x
NEW['took_time']=abs(NEW['modified_listing_date']-NEW['modified_issue_date'])

In [ ]:
train_df = NEW[NEW['label'] == 'train']
test_df = NEW[NEW['label'] == 'test']

In [ ]:
train_df_p = train_df.drop(['label','color_type','modified_listing_date','modified_issue_date'], axis=1)
test_df_p = test_df.drop(['label','color_type','modified_listing_date','modified_issue_date'], axis=1)

In [ ]:
train_df_b = pd.concat([train_df_p,train['pet_category']],axis='columns')
test_df_b = test_df.drop(['label','color_type','modified_listing_date','modified_issue_date'], axis=1)


## BayesianSearch CV

In [ ]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [ ]:
# Converting average precision score into a scorer suitable for model selection
roc_auc = make_scorer(f1_score, average='weighted',greater_is_better=True)

In [ ]:
# Setting a 5-fold stratified cross-validation (note: shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
# Initializing a CatBoostClassifier
c = CatBoostClassifier(thread_count=2, verbose = False)
x = XGBClassifier()
l = LGBMClassifier()


In [ ]:
# Defining your search space
cat = {'iterations': Integer(100, 500),
                 'depth': Integer(1, 10),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),}
               

# Defining your search space
xgb = {'max_depth': np.arange(5, 40),
                 'subsample': np.arange(0.1,1,0.1),
                 'min_child_weight': np.arange(1, 50),
                 'reg_lambda': np.arange(1, 10),
                 'reg_alpha': np.arange(1, 10),
                 'n_estimators': np.arange(10, 500,10),
                 'learning_rate':  np.arange(0.01, 0.1,0.01)}

lgm = {'max_depth': np.arange(5, 40),
                 'subsample': np.arange(0.1,1,0.1),
                 'min_child_weight': np.arange(0.00, 50),
                 'reg_lambda': np.arange(1, 10),
                 'reg_alpha': np.arange(1, 10),
                 'n_estimators': np.arange(10, 500,10),
                 'learning_rate':  np.arange(0.01, 0.1,0.01)}


In [ ]:
opt = BayesSearchCV(c,
                    cat,
                    scoring=roc_auc,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)

In [ ]:
best_params = report_perf(opt, train_df_p.iloc[:,3:], y_train2,'catboost', callbacks=[VerboseCallback(100), DeadlineStopper(60*10)])

In [ ]:
best_params = report_perf(opt, train_df_b.iloc[:,3:], y_train1,'catboost', callbacks=[VerboseCallback(100), DeadlineStopper(60*10)])

## XGBoost Classifier for Pet Category

('learning_rate', 0.05),
             ('max_depth', 37),
             ('min_child_weight', 6),
             ('n_estimators', 130),
             ('reg_alpha', 2),
             ('reg_lambda', 5),
             ('subsample', 0.2)])

[('learning_rate', 0.08),
             ('max_depth', 35),
             ('min_child_weight', 15),
             ('n_estimators', 470),
             ('reg_alpha', 8),
             ('reg_lambda', 1),
             ('subsample', 0.2)]

('learning_rate', 0.08),
             ('max_depth', 35),
             ('min_child_weight', 15),
             ('n_estimators', 470),
             ('reg_alpha', 8),
             ('reg_lambda', 1),
             ('subsample', 0.2)]                          

In [ ]:
X_train2, X_test2, y_train2_, y_test2 = train_test_split(train_df_p.iloc[:,3:], y_train2, test_size=0.20,random_state=44,stratify=y_train2)

clf1 = XGBClassifier().fit(X_train2, y_train2_)
y_pred2 = clf1.predict(X_test2)
print(sklearn.metrics.f1_score(y_test2,y_pred2,average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.898223457407381


In [ ]:
pet = clf1.predict(test_df_p.iloc[:,3:])
test_df_b['pet_category'] = pet

## XGBoost Classifier for Breed Category

[('learning_rate', 0.05),
             ('max_depth', 37),
             ('min_child_weight', 6),
             ('n_estimators', 220),
             ('reg_alpha', 2),
             ('reg_lambda', 5),
             ('subsample', 0.2)]
('learning_rate', 0.05),
             ('max_depth', 37),
             ('min_child_weight', 6),
             ('n_estimators', 220),
             ('reg_alpha', 2),
             ('reg_lambda', 5),
             ('subsample', 0.2)        

                

In [ ]:
X_train1, X_test1, y_train1_, y_test1 = train_test_split(train_df_b.iloc[:,3:], y_train1, test_size=.20,random_state=44,stratify=y_train1)

clf = XGBClassifier().fit(X_train1, y_train1_)
y_pred1 = clf.predict(X_test1)
print(sklearn.metrics.f1_score(y_test1,y_pred1,average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9071981144838769


## CatBoost Classifier for Pet and Breed Category

In [ ]:
X_train2, X_test2, y_train2_, y_test2 = train_test_split(train_df_p.iloc[:,3:], y_train2, test_size=0.20,random_state=44,stratify=y_train2)

clf1 = CatBoostClassifier(bagging_temperature= 0.7995534416071731,
             border_count = 112,
             depth = 6,
             iterations = 386,
             l2_leaf_reg = 27,
             learning_rate = 0.27168289873702833,
             random_strength = 1.744940909365023e-05,verbose=0).fit(X_train2, y_train2_)
y_pred2 = clf1.predict(X_test2)
print(sklearn.metrics.accuracy_score(y_test2,y_pred2)*100)

90.54950889301831


In [ ]:
pet = clf1.predict(test_df_p.iloc[:,3:])
test_df_b['pet_category'] = pet

In [ ]:
X_train1, X_test1, y_train1_, y_test1 = train_test_split(train_df_b.iloc[:,3:], y_train1, test_size=.20,random_state=44,stratify=y_train1)

clf = CatBoostClassifier(bagging_temperature= 0.41010395885331385,
             border_count = 186,
             depth = 9,
             iterations = 266,
             l2_leaf_reg = 21,
             learning_rate =  0.0673344419215237,
             random_strength =3.230824361824754e-06,verbose=0).fit(X_train1, y_train1_)
y_pred1 = clf.predict(X_test1)
print(sklearn.metrics.accuracy_score(y_test1,y_pred1)*100)

90.73533315635784


## LightGBM

[('bagging_temperature', 0.9732984017054159),
             ('border_count', 253),
             ('colsample_bylevel', 0.7844325590226278),
             ('depth', 1),
             ('iterations', 499),
             ('l2_leaf_reg', 4),
             ('learning_rate', 0.09837750765695877),
             ('random_strength', 8.465671519617828e-09)

In [ ]:
X_train2, X_test2, y_train2_, y_test2 = train_test_split(train_df_p.iloc[:,3:], y_train2, test_size=0.20,random_state=44,stratify=y_train2)

clf1 = LGBMClassifier().fit(X_train2, y_train2_)
y_pred2 = clf1.predict(X_test2)
print(sklearn.metrics.accuracy_score(y_test2,y_pred2))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9073533315635784


In [ ]:
pet = clf1.predict(test_df_p.iloc[:,3:])
test_df_b['pet_category'] = pet

[('bagging_temperature', 0.9555156939606648),
             ('border_count', 2),
             ('colsample_bylevel', 0.06842400861393462),
             ('depth', 9),
             ('iterations', 115),
             ('l2_leaf_reg', 24),
             ('learning_rate', 0.010100697732652058),
             ('random_strength', 5.0528974362458925)]

In [ ]:
X_train1, X_test1, y_train1_, y_test1 = train_test_split(train_df_b.iloc[:,3:], y_train1, test_size=.20,random_state=44,stratify=y_train1)

clf = LGBMClassifier().fit(X_train1, y_train1_)
y_pred1 = clf.predict(X_test1)
print(sklearn.metrics.accuracy_score(y_test1,y_pred1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9031059198301036


## F1-score(local)

In [ ]:
s1=f1_score(y_test1,y_pred1,average='weighted')
s2=f1_score(y_test2,y_pred2,average='weighted')
accuracy=100*((s1+s2)/2)
accuracy

90.79118686141672

## Submission file

In [ ]:
a = test_df['pet_id']
y = clf.predict(test_df_b.iloc[:,3:])
z = clf1.predict(test_df_p.iloc[:,3:])

In [ ]:
Prediction = pd.DataFrame({"pet_id":a , "breed_category":y.ravel(),"pet_category":z.ravel()})
Prediction.to_csv("Pet adoption new.csv",index=False)
files.download('Pet adoption new.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>